In [1]:
import tensorflow as tf
from pprint import pprint

In [2]:
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import *
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_accuracy
from keras.preprocessing.sequence import pad_sequences
from keras_bert import load_trained_model_from_checkpoint
from keras_bert import Tokenizer
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [3]:
label = {}
_label = {}
max_seq_length = 128
batch_size = 64
epochs = 10
lstmDim = 64
# model = self.Model()

## tag dict

In [4]:
label_path = "E://bert-model//tag_dict.txt"
f_label = open(label_path, 'r+', encoding='utf-8')
for line in f_label:
    content = line.strip().split()
    label[content[0].strip()] = content[1].strip()
    _label[content[1].strip()] = content[0].strip()

In [5]:
print( label )
print( _label )

{'O': '0', 'B': '1', 'I': '2', 'S': '3', 'E': '4'}
{'0': 'O', '1': 'B', '2': 'I', '3': 'S', '4': 'E'}


## vocab

In [6]:
vocab = {}
vocab_path = r"E:\Project\uncased_L-2_H-128_A-2\vocab.txt"
with open(vocab_path, 'r+', encoding='utf-8') as f_vocab:
    for line in f_vocab.readlines():
        vocab[line.strip()] = len(vocab)

In [7]:
def Id2Label( ids ):
    result = []
    for i in ids:
        result.append(_label[str(i)])
    return result

In [8]:
def Vector2Id( tags ):
    result = []
    for tag in tags:
        result.append(np.argmax(tag))
    return result

In [9]:
import pickle

with open('train_set_new','rb') as out_data:
    train_set = pickle.load(out_data)
    
with open('test_set_new','rb') as out_data:
    test_set = pickle.load(out_data)

## sample data

In [11]:
print( train_set[0] )

[['Experience', 'with', 'at', 'least', 'one', 'data', 'science', 'framework', 'such', 'as', 'R', ',', 'SAS', 'or', 'SPSS', '.'], ['O', 'O', 'O', 'O', 'O', 'S', 'E', 'E', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]


In [12]:
print( test_set[0] )

['O', 'O', 'O', 'O', 'O', 'S', 'E', 'E', 'O', 'O', 'B', 'O', 'B', 'O', 'B', 'O']


## building dataset

In [13]:
sentence_list = []
mask_list  = []
label_list = []
outout_list = []

train_set_new = []
test_set_new = []
for index in range( len( train_set ) ):
#     if len( train_set[index] ) >= 128:
#         print('Error')
        
#     else:
    if len( train_set[index] ) < 128 and len(set(test_set[index])) > 3:
        
        train_set_new.append( train_set[index] )
        test_set_new.append( test_set[index] )
        print('-'*120)
        my_tem_sentence_list = []
        for t in train_set[index][0]:
            my_tem_sentence_list.append( vocab.get(t.lower(),101) )

        my_tem_mask_list = []
        my_tem_mask_list = [1] * len( train_set[index][0] ) + [0] * (128 - len( train_set[index][0] ))
        my_tem_sentence_list = my_tem_sentence_list + [0] * (128 - len( train_set[index][0] ))

        print( train_set[index][0] )
        print( my_tem_sentence_list )
        sentence_list.append( my_tem_sentence_list )
        print()

        print( my_tem_mask_list )
        mask_list.append( my_tem_mask_list )
        print()

        my_tem_label_list  = []
        for t in train_set[index][1]:
            my_tem_label_list.append(  int( label.get(t,'0') ) ) #1

        my_tem_label_list = my_tem_label_list + [0] * (128 -  len( train_set[index][1] ) )
        print( train_set[index][1] )
        print( my_tem_label_list )
        label_list.append( my_tem_label_list )
        print( set( train_set[index][1] ) )

        print()

        my_tem_outout_list = []
        for t in test_set[index]:
            my_tem_outout_list.append( int( label.get(t,'0') ) ) #1
            
        my_tem_outout_list = my_tem_outout_list + [0] * (128 -  len( test_set[index] ) )

        print( test_set[index] )
        print( my_tem_outout_list )
        outout_list.append( my_tem_outout_list )

        print()

------------------------------------------------------------------------------------------------------------------------
['Experience', 'with', 'at', 'least', 'one', 'data', 'science', 'framework', 'such', 'as', 'R', ',', 'SAS', 'or', 'SPSS', '.']
[3325, 2007, 2012, 2560, 2028, 2951, 2671, 7705, 2107, 2004, 1054, 1010, 21871, 2030, 101, 1012, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,


[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

['O', 'O', 'O', 'O', 'O', 'O', 'S', 'E', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
[0, 0, 0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
{'S', 'O', 'E'}

['O', 'O', 'O', 'O', 'O', 'O', 'S', 'E', 'O', 'O', 'O', 'O', 'B', 'O', 'B', 'O', 'O', 'B']
[0, 0, 0, 0, 0, 0, 3, 4, 0, 0

In [24]:
print( len(sentence_list) )

85


In [15]:
sentence_list_train = []
mask_list_train  = []
label_list_train = []
outout_list_train = []


sentence_list_test = []
mask_list_test  = []
label_list_test = []
outout_list_test = []


for index in range( len( sentence_list ) ):
    if index % 5 != 0:
        sentence_list_train.append( sentence_list[index] )
        mask_list_train.append( mask_list[index] )
        label_list_train.append( label_list[index] )
        outout_list_train.append( outout_list[index] )
    else:
        sentence_list_test.append( sentence_list[index] )
        mask_list_test.append( mask_list[index] )
        label_list_test.append( label_list[index] )
        outout_list_test.append( outout_list[index] )
        
print( len( sentence_list_train ) )
print( len( sentence_list_test ) )

68
17


## recognize Subordinate word by sequence model

In [16]:
model_path = r"E:/Project/uncased_L-2_H-128_A-2/"
bert = load_trained_model_from_checkpoint(
    model_path + "bert_config.json",
    model_path + "bert_model.ckpt",
    seq_len=128
    )
#make bert layer trainable
for layer in bert.layers:
    layer.trainable = False


x1 = Input(shape=(None,))
x2 = Input(shape=(None,))
x3 = Input(shape=(None,))

bert_out = bert([x1, x2])

mask_layer = Embedding(6, 4)(x3)
print( mask_layer.shape )

cc = concatenate([bert_out, mask_layer])
print( cc.shape )
print( type(cc) )

lstm_out = Bidirectional(LSTM(lstmDim,
                                 return_sequences=True,
                                 dropout=0.2,
                                 recurrent_dropout=0.2))( cc )

crf_out = CRF(len(label), sparse_target=True)( lstm_out )
model = Model([x1, x2, x3], crf_out)
model.summary()

model.compile(
    optimizer=Adam(1e-4),
    loss=crf_loss,
    metrics=[crf_accuracy]
)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.







(?, ?, 4)
(?, ?, 132)
<class 'tensorflow.python.framework.ops.Tensor'>
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, None)         0                                            
________________

sentence_list_train = []
mask_list_train  = []
label_list_train = []
outout_list_train = []


sentence_list_test = []
mask_list_test  = []
label_list_test = []
outout_list_test = []

In [22]:
history = model.fit( x =[sentence_list_train, mask_list_train, label_list_train],
#                y = np.array( [ np.expand_dims(l,1) for l in outout_list] ),
#                y = outout_list,
               y = [[ np.expand_dims(l,-1) for l in outout_list_train]],
                    
               batch_size = 34,
               epochs=300,
               verbose=1,
               class_weight = 'auto')

model.save('keras_bert_mask')



Epoch 1/300
68/68 [==============================] - 4s 53ms/step - loss: 10.9814 - crf_accuracy: 0.1988
Epoch 2/300
68/68 [==============================] - 1s 14ms/step - loss: 10.9337 - crf_accuracy: 0.2204
Epoch 3/300
68/68 [==============================] - 1s 14ms/step - loss: 10.8699 - crf_accuracy: 0.2516
Epoch 4/300
68/68 [==============================] - 1s 14ms/step - loss: 10.8349 - crf_accuracy: 0.2625
Epoch 5/300
68/68 [==============================] - 1s 15ms/step - loss: 10.7751 - crf_accuracy: 0.2998
Epoch 6/300
68/68 [==============================] - 1s 15ms/step - loss: 10.7203 - crf_accuracy: 0.3756
Epoch 7/300
68/68 [==============================] - 1s 14ms/step - loss: 10.6994 - crf_accuracy: 0.3906
Epoch 8/300
68/68 [==============================] - 1s 13ms/step - loss: 10.6620 - crf_accuracy: 0.4490
Epoch 9/300
68/68 [==============================] - 1s 14ms/step - loss: 10.6181 - crf_accuracy: 0.5067
Epoch 10/300
68/68 [==============================] -

68/68 [==============================] - 1s 14ms/step - loss: 10.2291 - crf_accuracy: 0.6897
Epoch 75/300
68/68 [==============================] - 1s 14ms/step - loss: 10.2308 - crf_accuracy: 0.6900
Epoch 76/300
68/68 [==============================] - 1s 13ms/step - loss: 10.2177 - crf_accuracy: 0.6918
Epoch 77/300
68/68 [==============================] - 1s 13ms/step - loss: 10.2183 - crf_accuracy: 0.6858
Epoch 78/300
68/68 [==============================] - 1s 13ms/step - loss: 10.2168 - crf_accuracy: 0.6881
Epoch 79/300
68/68 [==============================] - 1s 13ms/step - loss: 10.2228 - crf_accuracy: 0.6899
Epoch 80/300
68/68 [==============================] - 1s 13ms/step - loss: 10.2218 - crf_accuracy: 0.6866
Epoch 81/300
68/68 [==============================] - 1s 13ms/step - loss: 10.2132 - crf_accuracy: 0.6907
Epoch 82/300
68/68 [==============================] - 1s 14ms/step - loss: 10.2174 - crf_accuracy: 0.6875
Epoch 83/300
68/68 [==============================] - 1s 13

68/68 [==============================] - 1s 12ms/step - loss: 10.0403 - crf_accuracy: 0.7108
Epoch 151/300
68/68 [==============================] - 1s 13ms/step - loss: 10.0383 - crf_accuracy: 0.7030
Epoch 152/300
68/68 [==============================] - 1s 15ms/step - loss: 10.0337 - crf_accuracy: 0.7116
Epoch 153/300
68/68 [==============================] - 1s 13ms/step - loss: 10.0341 - crf_accuracy: 0.7156
Epoch 154/300
68/68 [==============================] - 1s 16ms/step - loss: 10.0399 - crf_accuracy: 0.7070
Epoch 155/300
68/68 [==============================] - 1s 16ms/step - loss: 10.0329 - crf_accuracy: 0.7108
Epoch 156/300
68/68 [==============================] - 1s 16ms/step - loss: 10.0274 - crf_accuracy: 0.7168
Epoch 157/300
68/68 [==============================] - 1s 16ms/step - loss: 10.0191 - crf_accuracy: 0.7191
Epoch 158/300
68/68 [==============================] - 1s 15ms/step - loss: 10.0236 - crf_accuracy: 0.7119
Epoch 159/300
68/68 [==============================

68/68 [==============================] - 1s 13ms/step - loss: 9.8515 - crf_accuracy: 0.7762
Epoch 228/300
68/68 [==============================] - 1s 14ms/step - loss: 9.8555 - crf_accuracy: 0.7718
Epoch 229/300
68/68 [==============================] - 1s 14ms/step - loss: 9.8547 - crf_accuracy: 0.7751
Epoch 230/300
68/68 [==============================] - 1s 16ms/step - loss: 9.8589 - crf_accuracy: 0.7629
Epoch 231/300
68/68 [==============================] - 1s 14ms/step - loss: 9.8522 - crf_accuracy: 0.7699
Epoch 232/300
68/68 [==============================] - 1s 15ms/step - loss: 9.8564 - crf_accuracy: 0.7761
Epoch 233/300
68/68 [==============================] - 1s 17ms/step - loss: 9.8491 - crf_accuracy: 0.7635
Epoch 234/300
68/68 [==============================] - 1s 14ms/step - loss: 9.8423 - crf_accuracy: 0.7809
Epoch 235/300
68/68 [==============================] - 1s 15ms/step - loss: 9.8456 - crf_accuracy: 0.7711
Epoch 236/300
68/68 [==============================] - 1s 15

## output test result

In [23]:
for i in range( len(sentence_list) ):
    
    print( '-' * 120)
    print( train_set_new[i][0] )
    print( train_set_new[i][1] )
    print( test_set_new[i] )

    tags = model.predict([sentence_list[i:i+1], mask_list[i:i+1], label_list[i:i+1]])[0]

    result = []
    for j in range( len(train_set_new[i][0]) ):
        result.append(tags[j])
    result = Vector2Id(result)
    tag = Id2Label(result)

#     if len( set(tag) ) > 1:
    print('result:')
    print(tag)

    print( str(test_set_new[i]) == str(tag) )

------------------------------------------------------------------------------------------------------------------------
['Experience', 'with', 'at', 'least', 'one', 'data', 'science', 'framework', 'such', 'as', 'R', ',', 'SAS', 'or', 'SPSS', '.']
['O', 'O', 'O', 'O', 'O', 'S', 'E', 'E', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['O', 'O', 'O', 'O', 'O', 'S', 'E', 'E', 'O', 'O', 'B', 'O', 'B', 'O', 'B', 'O']
result:
['O', 'O', 'O', 'O', 'O', 'S', 'E', 'O', 'O', 'O', 'B', 'O', 'O', 'O', 'B', 'O']
False
------------------------------------------------------------------------------------------------------------------------
['Experience', 'with', 'Hadoop', 'stack', '(', 'HIVE', ',', 'Pig', ',', 'Hadoop', 'streaming', ')', 'and', 'Spark', 'a', 'plus']
['O', 'O', 'S', 'E', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['O', 'O', 'S', 'E', 'O', 'B', 'O', 'B', 'O', 'B', 'I', 'O', 'O', 'O', 'O', 'O']
result:
['O', 'O', 'O', 'E', 'O', 'B', 'O', 'B', 'O', 'B', 'B', 'O', 'O', 'B', 'O',

result:
['O', 'O', 'O', 'O', 'O', 'S', 'E', 'O', 'O', 'B', 'O', 'B', 'O', 'B', 'O', 'B', 'O']
True
------------------------------------------------------------------------------------------------------------------------
['Experience', 'with', 'statistical', 'programming', 'languages', 'like', 'SAS', ',', 'Excel', ',', 'and', 'R']
['O', 'O', 'O', 'S', 'E', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['O', 'O', 'O', 'S', 'E', 'O', 'B', 'O', 'B', 'O', 'O', 'B']
result:
['O', 'O', 'S', 'S', 'E', 'O', 'B', 'O', 'B', 'O', 'O', 'B']
False
------------------------------------------------------------------------------------------------------------------------
['Extensive', 'experience', 'with', 'advanced', 'analytics', 'tools', 'such', 'as', 'R', ',', 'Python', ',', 'CPLEX', ',', 'XPRESS', ',', 'SQL', ',', 'MATLAB', ',', 'etc', '.']
['O', 'O', 'O', 'S', 'E', 'E', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['O', 'O', 'O', 'S', 'E', 'E', 'O', 'O', 'B', 'O', 'B', 'O', '

result:
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'O', 'B', 'O', 'B', 'O', 'B', 'O', 'O', 'O']
False
------------------------------------------------------------------------------------------------------------------------
['Hadoop', 'stack', '(', 'HIVE', ',', 'Pig', ',', 'Hadoop', 'streaming', ')']
['S', 'E', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['S', 'E', 'O', 'B', 'O', 'B', 'O', 'B', 'I', 'O']
result:
['O', 'O', 'O', 'B', 'O', 'B', 'O', 'B', 'B', 'O']
False
------------------------------------------------------------------------------------------------------------------------
['3', '+', 'years', 'of', 'experience', 'in', 'a', 'programming', 'language', 'such', 'as', 'Python', ',', 'R', ',', 'Java']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'S', 'E', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'S', 'E', 'O', 'O', 'B', 'O', 'B', 'O', 'B']
result:
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'E', 'O', 'O', 'B', 'O', 'B', 'O', 'B']
False
---------------------

result:
['O', 'O', 'O', 'O', 'S', 'E', 'O', 'O', 'B', 'O', 'B', 'B', 'O', 'B', 'O', 'B', 'O', 'O', 'O']
False
------------------------------------------------------------------------------------------------------------------------
['Minimum', '8', 'yr', ' ', 'Years', 'Statistical', 'modeling', 'like', 'linear', ',', 'logistic', 'regression', ',', 'classification', ',', 'hypothesis', 'testing', ',', 'ANOVA', ',', 'PCA', ',', 'SVD', ',', 'etc', '.']
['O', 'O', 'O', 'O', 'S', 'E', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['O', 'O', 'O', 'O', 'S', 'E', 'O', 'B', 'O', 'B', 'I', 'O', 'B', 'O', 'B', 'I', 'O', 'B', 'O', 'B', 'O', 'B', 'O', 'O', 'O']
result:
['O', 'O', 'O', 'O', 'O', 'O', 'E', 'O', 'B', 'O', 'B', 'B', 'O', 'B', 'O', 'B', 'B', 'O', 'B', 'O', 'B', 'O', 'B', 'O', 'O', 'O']
False
------------------------------------------------------------------------------------------------------------------------
['Experience', 'in', 'analysis

result:
['O', 'O', 'S', 'E', 'O', 'O', 'O', 'B', 'B', 'O', 'B', 'O', 'O', 'O', 'O', 'O', 'O', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'O', 'B', 'O', 'O', 'O', 'O', 'B', 'O', 'O', 'O']
False
------------------------------------------------------------------------------------------------------------------------
['Experience', 'with', 'sophisticated', 'analyses', '(', 'like', ',', 'longitudinal', 'modeling', ',', 'machine', 'learning', ',', 'non', '-', 'parametric', 'statistical', 'modeling', ')', 'and', 'data', 'visualization', '(', 'like', ',', 'Tableau', ',', 'Qlik', ',', 'R', 'Shiny', ',', 'Microstrategy', ')', 'is', 'preferred']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'S', 'E', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'S', 'E', 'O', 'O', 'O', 'B', 'O', 'B', 'O', 'B', 'I', 'O', 'B', 'O', 'O',

result:
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'E', 'O', 'O', 'O', 'B', 'O', 'B', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'O', 'O', 'O', 'O', 'B', 'O', 'B', 'O', 'B', 'O']
False
------------------------------------------------------------------------------------------------------------------------
['Knowledge', 'of', 'all', 'phases', 'of', 'software', 'development', 'including', 'design', ',', 'coding', ',', 'testing', ',', 'debugging', ',', 'implementation', ',', 'and', 'support']
['O', 'O', 'O', 'O', 'O', 'S', 'E', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['O', 'O', 'O', 'O', 'O', 'S', 'E', 'O', 'B', 'O', 'B', 'O', 'B', 'O', 'B', 'O', 'B', 'O', 'O', 'B']
result:
['O', 'O', 'O', 'O', 'O', 'E', 'O', 'O', 'B', 'O', 'B', 'O', 'B', 'O', 'B', 'O', 'B', 'O', 'O', 'O']
False
------------------------------------------------------------------------------------------------------------------------
['Experience', 'with', 'relational', '(', 'SQL', 'Server', ',',